# Sub-classing Generic Views

En esta lectura, aprenderá sobre las vistas genéricas basadas en clases. Puede implementar la capa de vista con funciones o vistas basadas en clases en una aplicación Django. Declara una clase extendiendo la clase django.views.View y define los métodos get() y post() dentro de ella para manejar las solicitudes HTTP. El patrón de URL conecta la ruta a la clase con el método as_view() de la clase View. Un ejemplo rápido de vista de clase:

In [1]:
from django.views import View   
from django.shortcuts import HttpResponse

class NewView(View):   
    def get(self, request):   
        # View logic will place here   
        return HttpResponse('response') 

In [5]:
# El patrón de URL en urls.py se actualiza de la siguiente manera:

#urls.py
#from myapp.views import NewView   
from django.urls import path, include
urlpatterns = [   
    path('about/', NewView.as_view()),   
]  

Django proporciona vistas genéricas basadas en clases para que el proceso de desarrollo sea mucho más fácil y rápido. En esta lectura, descubrirá cómo se implementan las vistas genéricas. Las vistas genéricas más utilizadas son:

- TemplateView

- CreateView 

- ListView

- DetailView 

- UpdateView 

- DeleteView

- LoginView

## Diferencia entre vista de función y vista genérica 

Tomemos un ejemplo simple de representación de una plantilla Hello world. 

Debe representar la plantilla como la respuesta HTTP: el valor de retorno de la función. La función de vista para este propósito sería la siguiente:

In [18]:
from django.shortcuts import render 
from django.template import loader

def index(request): 
    template = loader.get_template('myapp/index.html') 
    context={} 
    return HttpResponse(template.render(context, request)) 

Como ya habrá aprendido, la carpeta del paquete de la aplicación existe dentro de la carpeta del contenedor externo del proyecto. La aplicación myapp se crea en el proyecto Django denominado myproject.

In the myapp/urls.py, define the URL pattern:

```python 

path('/', views.index, name='index') 

```

Utilice una clase TemplateView en su lugar. Simplemente defina una clase que la amplíe y establezca el atributo template_name.

In [ ]:
from django.views.generic.base import TemplateView 

class IndexView(TemplateView): 
    template_name = 'index.html' 

The corresponding URL pattern should incorporate this:

```python 
from . import views

path('/', views.IndexView.as_view(), name='index') 

```

Ventaja de las vistas basadas en funciones 

- Simple de implementar. 
- Fácil de leer. 
- Flujo de código explícito. 
- Uso sencillo de decoradores. 
- bueno para una funcionalidad única o especializada. 

Desventajas de las vistas basadas en funciones 

- Difícil de extender y reutilizar el código. 
- Manejo de métodos HTTP a través de bifurcaciones condicionales. 

Ventajas de las vistas basadas en clases 
 
- Reutilización de código. 
- DRY: el uso de CBV ayuda a reducir la duplicación de código. 
- El código se puede ampliar para incluir más funcionalidades. 
- clase con diferentes métodos para cada solicitud http en lugar de declaraciones de bifurcación condicional dentro de una sola vista basada en funciones. 
- Vistas genéricas basadas en clases integradas. 

Desventajas de las vistas basadas en clases 

- Más difícil de leer. 
- Flujo de código implícito. 
- El uso de decoradores de vista requiere una importación adicional o una anulación de método. 

Requisitos de una vista genérica 

- Si la vista necesita que se procese un modelo, debe establecerse como el valor de la propiedad del modelo de la vista. 
- Cada tipo de vista busca un nombre de plantilla con nombre de modelo con el sufijo del tipo de vista genérica. Por ejemplo, para un modelo de empleado de procesamiento de vista de lista, Django intenta encontrar employee_list.html. 
- La vista genérica se asigna a la URL con el método as_view() de la clase View.

Ahora construyamos una subclase para cada una de las respectivas clases de vista genérica para realizar operaciones CRUD en el modelo de empleado:

```python

from django.db import models

class Employee(models.Model):   
    name    = models.CharField(max_length=100)   
    email   = models.EmailField()   
    contact = models.CharField(max_length=15)   
    class Meta:   
        db_table = "Employee" 
```

## CreateView
LA Clase CreateView  automatiza la creación de una nueva instancia del modelo. Para proporcionar una vista de creación, use la subclase de  CreateView:

```python
from django.views.generic.edit import CreateView   

class EmployeeCreate(CreateView):   
    model = Employee    
    fields = '__all__' 
    success_url = "/employees/success/" 
```
Esta vista crea un formulario modelo basado en la estructura del modelo y lo pasa a la plantilla employeeCreate.html:

```html
<form method="post"> 
{% csrf_token %} 
<table> 
    {{ form.as_table }} 
</table> 
    <input type="submit" value="Save"> 
</form> 
```

La ruta URL se actualiza asignando la ruta "create/" al método as_view() de esta clase:

```python

from .views import EmployeeCreate 
urlpatterns = [ 
    . . . 
    path('create/', EmployeeCreate.as_view(), name = 'EmployeeCreate')  , 
] 
```

Cuando el cliente visita esta URL, se le presenta el formulario. El usuario completa y envía los detalles del empleado, que se guardan en la tabla Empleado.




# ListView

El módulo django.views.generic.list de Django contiene la definición de la clase ListView. Escriba su subclase para representar la lista de objetos modelo. 

La clase EmployeeList es similar a la subclase CreateView, excepto por su clase base.

```python
from django.views.generic.list import ListView  
class EmployeeList(ListView):   
    model = Employee   
    success_url = "/employees/success/" 
```
La plantilla requerida para esta vista debe llamarse employee_list.html. Django envía el objeto modelo en su contexto. Usando la sintaxis de bucle DTL, puede mostrar la lista de empleados:

```html
<ul>   
        {% for object in object_list %}   
        <li>Name: {{ object.name }}</li>   
        <li>Email: {{ object.email }}</li>   
        <li>contact: {{ object.contact }}</li>  
        <br/> 
        {% endfor %}   
</ul> 
```

If the user visits 
http://localhost:/8000/employees/list
, the browser lists all the rows in the employee table.


El DetailView genérico se encuentra en el módulo django.views.generic.detail. Ahora, cree su subclase, EmployeeDetail (de la misma manera que EmployeeList). 

```python
from django.views.generic.detail import DetailView  
class EmployeeList(DetailView):   
    model = Employee   
    success_url = "/employees/success/" 
```

Tenga en cuenta que esta vista muestra los detalles de un objeto cuya clave principal se pasa como argumento en la URL. Por lo tanto, agregue la siguiente ruta en el patrón de URL de la aplicación:

```python
path('show/<int:pk>', EmployeeDetail.as_view(), name = 'EmployeeDetail') 
```

Una vez más, el objeto View obtiene la instancia del modelo y la pasa como contexto a la plantilla employee_detail.html, que muestra sus atributos con la sintaxis de plantilla de la siguiente manera:

```html
<h1>Name : {{object.name}}</h1>   

    <p>Email : {{ object.email }}</p>   
    <p>Contact : {{ object.contact }}</p> 
```

Assuming the URL visited is /employees/1, the Employee record with primary key=1 will be displayed.



## UpdateView

Dado que esta vista está destinada a actualizar los datos de una instancia de modelo dada, su ruta URL debe configurarse en consecuencia:

```python
from django.views.generic.edit import UpdateView  
class EmployeeUpdate(UpdateView):   
    model = Employee   
    fields = '__all__'   
    success_url = "/employees/success/" 
```

Dado que esta vista está destinada a actualizar los datos de una instancia de modelo dada, su ruta URL debe configurarse en consecuencia:

```python
path('update/<int:pk>', EmployeeUpdate.as_view(), name = 'EmployeeUpdate')
```

The example URL that invokes this view is /employees/1.

The UpdateView class renders the template with its name having update_formas suffix:

#employee_update_form.html 

```html
<form method="post"> 
{% csrf_token %} 
<table> 
    {{ form.as_table }} 
</table> 
    <input type="submit" value="Save"> 
</form> 
```

## DeleteView

Por último, la vista genérica realiza la operación de eliminación en la instancia dada de un modelo. Está presente al editar el submódulo del módulo django.views.generic:

```python
from django.views.generic.edit import DeleteView 
class EmployeeDelete(DeleteView):   
    model = Employee   
    success_url = "/employees/success/" 

# urls.py
path('<delete/int:pk>', EmployeeDelete.as_view(), name = 'EmployeeDelete')
```

La plantilla employee_confirm_delete.html solicita la confirmación del usuario antes de eliminar la instancia del modelo.

```html
<form method="post"> 
{% csrf_token %}   

    <p>Are you sure you want to delete "{{ object }}"?</p>   

    <input type="submit" value="Confirm">   
</form> 
```
En esta lectura, conoció las vistas genéricas basadas en clases para realizar operaciones CRUD. Las vistas genéricas son mucho más fáciles de escribir que la vista basada en funciones, aunque ambas tienen sus propias ventajas y desventajas.

